# LAB2

In lab2, we will guide you through the basic Scallop Python API, called `scallopy`.
In this tutorial, you will learn:
1. How to construct and execute a Scallop program in Python using `scallopy`.
2. How to perform a learning task in Python through `scallopy`.

## Hello Scallopy

Let's write our first hello world Python program. You can do this purely through the scallopy interface. 

In [ ]:
# We start from constructing a scallopy context
import scallopy
ctx = scallopy.ScallopContext()

# We declare a relation type using 'add_relation'. 
# This is equvalent to 'type hello(String)' in a .scl file
ctx.add_relation("hello", str)

# We add the fact hello("Hello World") to the scallopy context 
ctx.add_facts("hello", [("Hello World",)])

# We can execute the context through 'run'
ctx.run()
print(list(ctx.relation("hello")))


## MNIST

![img/mnist_example.png](https://github.com/scallop-lang/scallop-lang.github.io/blob/master/img/summer_school/lab2/mnist_example.png?raw=true)

### P1: Count How Many 2

Let's first construct the symbolic representation of the MNIST images shown above.
You should use the function `ctx.add_relation(RELATION_NAME, TUPLE_OF_TYPES)` to declare the type of relations,
and use the function `ctx.add_facts(RELATION_NAME, TUPLES)` to add a list of tuples into the relation `RELATION_NAME`.
For `TUPLE_OF_TYPES`, you can use `(int, int)` to represent a tuple of two integers, where `int` is idiomatic Python type annotation.
The expected input facts for `digit(i, d)` should represent the mapping from image ID `i` to its numerical digit `d`.
Note that in order for any fact to be added to the context, you have to declare the type of that relation beforehands.

Then, please use the function `ctx.add_rule(RULE_STRING)` to add a rule that counts how many `2` are there in the image.
This rule should start with `"num_of_2(n) = FILL_IN_YOUR_RULE_BODY"`.
Note that we have omitted the `rel` keyword before the rule, 
and you don't need to declare the type of the relation `num_of_2` beforehand since `scallopy` can infer the type from the rule.

At the end, you should be able to see `[(1,)]` being printed out:
The relation `num_of_2` is a set of unary tuples and there is only one tuple containing the count of 2.
The tuple contains `1` suggesting that there is only one `2` among the digits.

In [ ]:
import os
import scallopy

ctx = scallopy.ScallopContext()

# TODO: 1. Declare the relation `digit`
# TODO: 2. Add facts into the relation `digit`
# TODO: 3. Add a rule that counts the number of `2`s from the given digits. Populate a relation `num_of_2`

ctx.run()
print(list(ctx.relation("num_of_2"))) # Should be printing [(1,)]

### P2: Count How Many Less Than 5, Probabilistic

Let's try to write a probabilistic symbolic representation of the MNIST images.
Similar to the previous problem, please add the relation and facts for the relation `digit`.
However, when adding the facts this time, let's do it with probabilities!

``` python
ctx.add_fact(RELATION_NAME, [(PROB, TUPLE), (PROB, TUPLE), ...])
```

Now, since we have the full Python syntax at hand, you are free to use `random()` to generate `[0, 1]` floating point numbers,
for loop and even list comprehensions.
If you choose to randomly generate probabilities, make sure you have the probability distribution for each digit sum up to 1.


In [ ]:
import os
from random import random
import scallopy

provenance = "FILL_IN_YOUR_PROVENANCE"
provenance = "minmaxprob"
ctx = scallopy.ScallopContext(provenance=provenance)

### P3: MNIST Sum 2

The whole purpose of this experiment is for you to see through the MNIST Sum 2 Experiment!
We are showing all the code needed to actually run the experiment, separated into chunks.
There are parts you don't need to pay much attention and parts that you want to understand a few functions.

The block below just imports dependencies and setup some pre-requisites.

In [ ]:
import os
import random
from typing import *
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

mnist_img_transform = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize(
    (0.1307,), (0.3081,)
  )
])

#### MNIST Sum 2 Dataset

The following class `MNISTSum2Dataset` defines a dataset.
A PyTorch dataset needs to implement two functions, `__len__()` (length) and `__getitem__(index: int)`.

In our case, we base our dataset on the original MNIST dataset provided by `torchvision`.
We randomize the source dataset by creating a suffled index map.
And since the resulting input will be two images, we will pull two data-points from the MNIST dataset.
At the end, each data-point will have three things: the two input images and the sum of the two digits (integer).

There is also a `collate_fn` that collate data-points into batches.
The input `batch` is a list of tuples `(a_img, b_img, a_digit + b_digit)`.
It will return tuple of three batches of elements: `((a_imgs, b_imgs), digits)`.
Note that there are only two elements in this tuple since we want each data-point to be separated into input `x` and ground-truth `y`.
Inside of it, we use `torch.stack(LIST_OF_TENSORS)` to "stack" images into batch of images.
At the end, `a_imgs` and `b_imgs` will be two batches of 64 images (64 is the "batch size" as we call it),
and `digits` will be a batch of 64 numbers.

In [ ]:
class MNISTSum2Dataset(torch.utils.data.Dataset):
  def __init__(self, root: str, train: bool = True, transform: Optional[Callable] = None, target_transform: Optional[Callable] = None, download: bool = False):
    # Contains a MNIST dataset
    self.mnist_dataset = torchvision.datasets.MNIST(root, train=train, transform=transform, target_transform=target_transform, download=download)
    self.index_map = list(range(len(self.mnist_dataset)))
    random.shuffle(self.index_map)

  def __len__(self):
    return int(len(self.mnist_dataset) / 2)

  def __getitem__(self, idx):
    # Get two data points
    (a_img, a_digit) = self.mnist_dataset[self.index_map[idx * 2]]
    (b_img, b_digit) = self.mnist_dataset[self.index_map[idx * 2 + 1]]

    # Each data has two images and the GT is the sum of two digits
    return (a_img, b_img, a_digit + b_digit)

  @staticmethod
  def collate_fn(batch):
    a_imgs = torch.stack([item[0] for item in batch])
    b_imgs = torch.stack([item[1] for item in batch])
    digits = torch.stack([torch.tensor(item[2]).long() for item in batch])
    return ((a_imgs, b_imgs), digits)


#### The Dataloader for MNIST Sum 2

This is a wrapper outside of the `MNISTSum2Dataset` so that it produces shuffled and batched input and output pairs.
Not too interesting

In [ ]:
def mnist_sum_2_loader(data_dir, batch_size_train, batch_size_test):
  train_loader = torch.utils.data.DataLoader(
    MNISTSum2Dataset(data_dir, train=True, download=True, transform=mnist_img_transform),
    collate_fn=MNISTSum2Dataset.collate_fn,
    batch_size=batch_size_train,
    shuffle=True
  )
  test_loader = torch.utils.data.DataLoader(
    MNISTSum2Dataset(data_dir, train=False, download=True, transform=mnist_img_transform),
    collate_fn=MNISTSum2Dataset.collate_fn,
    batch_size=batch_size_test,
    shuffle=True
  )
  return train_loader, test_loader

#### MNIST Neural Network

This is the MNIST Neural Network based on Convolutional Neural Networks (CNN).
Inside of initialization, we have two convolutional layers and two linear layers.
During forward, we take in the input `x` (which should be a batch of images) and output a tensor of size 64 x 10 (notice that `self.fc2`, the last layer of the network, has an output size of 10).
At the end, we will do a `softmax` step to create a probabilistic distribution of 10 possibilities.

In [ ]:
class MNISTNet(nn.Module):
  def __init__(self):
    super(MNISTNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
    self.fc1 = nn.Linear(1024, 1024)
    self.fc2 = nn.Linear(1024, 10)

  def forward(self, x):
    x = F.max_pool2d(self.conv1(x), 2)
    x = F.max_pool2d(self.conv2(x), 2)
    x = x.view(-1, 1024)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, p = 0.5, training=self.training)
    x = self.fc2(x)
    return F.softmax(x, dim=1)

#### MNIST Sum 2 Model

This model combines the perception model (MNISTNet) and Sum 2 Scallop Program.
During initialization, we setup our `MNISTNet` for training.
Additionally, we setup the `ScallopContext`, which is a member of `scallopy`.
Note that the `provenance` is customizable and stated as an input to the `MNISTSum2Net`.
We add the relations `digit_1` and `digit_2`, and setup the `input_mapping`s, and add the rule of the `sum_2` relation.
Finally, we setup a forward function that takes the relation `sum_2` out and apply an `output_mapping` to turn it into a PyTorch tensor.

During `forward`, we apply `self.mnist_net` on both batches of images, and pass these into the `sum_2` forward function.
Note that there are named parameters so that `a_distrs` is passed into `"digit_1"` relation and `b_distrs` is passed into the `"digit_2"` relation.
At the end, this function will return a Batch Size x 19 tensor, since we have setup an `output_mapping` of size 19.

In [ ]:
class MNISTSum2Net(nn.Module):
  def __init__(self, provenance, k):
    super(MNISTSum2Net, self).__init__()

    # MNIST Digit Recognition Network
    self.mnist_net = MNISTNet()

    # Scallop Context
    self.scl_ctx = scallopy.ScallopContext(provenance=provenance, k=k)
    self.scl_ctx.add_relation("digit_1", int, input_mapping=list(range(10)))
    self.scl_ctx.add_relation("digit_2", int, input_mapping=list(range(10)))
    self.scl_ctx.add_rule("sum_2(a + b) :- digit_1(a), digit_2(b)")

    # The `sum_2` logical reasoning module
    self.sum_2 = self.scl_ctx.forward_function("sum_2", output_mapping=[(i,) for i in range(19)])

  def forward(self, x: Tuple[torch.Tensor, torch.Tensor]):
    (a_imgs, b_imgs) = x

    # First recognize the two digits
    a_distrs = self.mnist_net(a_imgs) # Tensor 64 x 10
    b_distrs = self.mnist_net(b_imgs) # Tensor 64 x 10

    # Then execute the reasoning module; the result is a size 19 tensor
    return self.sum_2(digit_1=a_distrs, digit_2=b_distrs) # Tensor 64 x 19

#### MNIST Sum 2 Trainer

This `MNISTSum2Trainer` class takes care of the training loop.
During initialization, we setup our `MNISTSum2Net` and setup our optimizer and two data loaders (for training and testing).
For a training epoch, we will pass the input `x` into the network and which will produce the output `y_pred`.
`y_pred` will be compared with `y` in a BCE loss function and the loss will be back-propagated into the neural networks.

We will start by running a `test_epoch` just to understand what the accuracy with no training is.
After that, we will alternate between `train_epoch` and `test_epoch` to see what improvement we have obtained for each epoch.

In [ ]:
class MNISTSum2Trainer:
  def __init__(self, train_loader, test_loader, learning_rate, k, provenance):
    self.network = MNISTSum2Net(provenance, k)
    self.optimizer = optim.Adam(self.network.parameters(), lr=learning_rate)
    self.train_loader = train_loader
    self.test_loader = test_loader

  def bce_loss(self, y_pred, y):
    (_, dim) = y_pred.shape
    gt = torch.stack([torch.tensor([1.0 if i == t else 0.0 for i in range(dim)]) for t in y])
    return F.binary_cross_entropy(y_pred, gt)
  
  def train_epoch(self, epoch):
    self.network.train()
    iter = tqdm(self.train_loader, total=len(self.train_loader))
    for (x, y) in iter:
      self.optimizer.zero_grad()
      y_pred = self.network(x)
      loss = self.bce_loss(y_pred, y)
      loss.backward()
      self.optimizer.step()
      iter.set_description(f"[Train Epoch {epoch}] Loss: {loss.item():.4f}")

  def test_epoch(self, epoch):
    self.network.eval()
    num_items = len(self.test_loader.dataset)
    test_loss = 0
    correct = 0
    with torch.no_grad():
      iter = tqdm(self.test_loader, total=len(self.test_loader))
      for (x, y) in iter:
        output = self.network(x)
        test_loss += self.bce_loss(output, y).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).sum()
        perc = 100. * correct / num_items
        iter.set_description(f"[Test Epoch {epoch}] Total loss: {test_loss:.4f}, Accuracy: {correct}/{num_items} ({perc:.2f}%)")

  def train(self, n_epochs):
    self.test_epoch(0)
    for epoch in range(1, n_epochs + 1):
      self.train_epoch(epoch)
      self.test_epoch(epoch)


Finally, try to run the following block, while tuning the parameters among
- `difftopkproofs`
- `diffminmaxprob`
- `diffaddmultprob`

And for `difftopkproofs`, please also tune the value of `k` and record the numbers in the following table:

| Iteration | difftop1proofs | difftop3proofs | difftop10proofs | diffminmaxprob | diffaddmultprob |
|-----------|----------------|----------------|-----------------|----------------|-----------------|
| Epoch 1   | ??% | ??% | ??% | ??% | ??% |
| Epoch 2   | ??% | ??% | ??% | ??% | ??% |

In [ ]:
def run_mnist_sum_2():
  data_dir = "."
  batch_size_train = 64
  batch_size_test = 64
  n_epochs = 2
  learning_rate = 0.001
  provenance = "difftopkproofs"
  k = 3

  train_loader, test_loader = mnist_sum_2_loader(data_dir, batch_size_train, batch_size_test)
  trainer = MNISTSum2Trainer(train_loader, test_loader, learning_rate, k, provenance)
  trainer.train(n_epochs)

run_mnist_sum_2()

### P4: MNIST Sum 3
In this practice, we will use scallopy to train an MNIST digit recognition network. Given three MNIST numbers and their sum, we want to train a classifier that can identify the digits, and yields a correct sum of the input images.

**Step 1** Dataloader construction. 

First, we want to construct a train data loader, and a test data loader separately. 
Please fill in the `get_item` and `collate_fn` functions for the dataloader.
The `get_item` function shall take in an index and return a tuple. The first tuple element is a triplet of tensorized images, and the second tuple element is the sum of the images.
The `collate_fn` function shall take in a list of tuples returned by `get_item`, and return a tuple. The first tuple element is a triplet of batched tensors representing the images, and the second element is a tensor of batched sum values. 

In [ ]:
class MNISTSum3Dataset(torch.utils.data.Dataset):
  def __init__(
    self,
    root: str,
    train: bool = True,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    download: bool = False,
  ):
    # Contains a MNIST dataset
    self.mnist_dataset = torchvision.datasets.MNIST(
      root,
      train=train,
      transform=transform,
      target_transform=target_transform,
      download=download,
    )
    self.index_map = list(range(len(self.mnist_dataset)))
    random.shuffle(self.index_map)

  def __len__(self):
    return int(len(self.mnist_dataset) / 3)

  # The `get_item` function shall take in an index and return a tuple. 
  # The first tuple element is a triplet of tensorized images, 
  # and the second tuple element is the sum of the images.
  def __getitem__(self, idx):
    # TODO: Complete the __getitem__ method
    raise NotImplementedError

  # The `collate_fn` function shall take in a list of tuples returned by `get_item`, 
  # and return a tuple. The first tuple element is triplet of batched tensors 
  # representing the images, and the second element is a tensor of batched sum values.  
  @staticmethod
  def collate_fn(batch):
    # TODO: complete the collate_fn method
    raise NotImplementedError

def mnist_sum_3_loader(data_dir, batch_size_train, batch_size_test):

  train_loader = torch.utils.data.DataLoader(
    MNISTSum3Dataset(
      data_dir,
      train=True,
      download=True,
      transform=mnist_img_transform,
    ),
    collate_fn=MNISTSum3Dataset.collate_fn,
    batch_size=batch_size_train,
    shuffle=True
  )

  test_loader = torch.utils.data.DataLoader(
    MNISTSum3Dataset(
      data_dir,
      train=False,
      download=True,
      transform=mnist_img_transform,
    ),
    collate_fn=MNISTSum3Dataset.collate_fn,
    batch_size=batch_size_test,
    shuffle=True
  )

  return train_loader, test_loader


You can take a look into the dataset with matplotlib. 

In [ ]:
import matplotlib.pyplot as plt
import torch, random

# Feel free to modify the parameters below
seed = 1234
batch_size_train = 64
batch_size_test = 64

torch.manual_seed(seed)
random.seed(seed)
data_dir = os.path.abspath(os.path.join(os.path.abspath("__file__"), "../data"))
train_loader, test_loader = mnist_sum_3_loader(data_dir, batch_size_train, batch_size_test)

# Let's take a look into the dataset
print(f"The dataset size is: {len(train_loader)}.")
for (x, y) in train_loader:
    # The dataloader will give you batches of three MNIST images and their sum 
    (a_imgs, b_imgs, c_imgs), digits = (x, y)
    print(a_imgs.shape)

    # We can peek the CLEVR image in the dataset
    imgplot = plt.imshow(a_imgs[0].reshape(28, 28), cmap='gray')
    plt.show()
    break

**Step 2** Construct a classifier `MNISTSum3Net` that takes in three MNIST images and returns a tensor of the distribution of their sum over 0 to 27.

In [ ]:
import scallopy
class MNISTSum3Net(nn.Module):
  def __init__(self, provenance, k):
    super(MNISTSum3Net, self).__init__()
    # TODO: Initialize the nueral network here. It should include:
    #       1. MNISTNet
    #       2. Scallop program
    #       3. Forward function
    raise NotImplementedError

  def forward(self, x):
    # TODO: Write the forward function for MNISTSum3Net
    # Then execute the reasoning module; the expected return value is a size 28 tensor
    raise NotImplementedError

**Step 3** Setup trainer. We will use the BCE loss function for training the model.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

def bce_loss(output, ground_truth):
  (_, dim) = output.shape
  gt = torch.stack([torch.tensor([1.0 if i == t else 0.0 for i in range(dim)]) for t in ground_truth])
  return F.binary_cross_entropy(output, gt)

class Trainer():
  def __init__(self, train_loader, test_loader, learning_rate, k, provenance):
    self.network = MNISTSum3Net(provenance, k)
    self.optimizer = optim.Adam(self.network.parameters(), lr=learning_rate)
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.loss = bce_loss

  def train_epoch(self, epoch):
    self.network.train()
    iter = tqdm(self.train_loader, total=len(self.train_loader))
    train_loss = 0
    correct = 0
    total = 0
    for data_ct, (data, target) in enumerate(iter):
      self.optimizer.zero_grad()
      output = self.network(data)

      loss = self.loss(output, target)
      loss.backward()
      self.optimizer.step()
      train_loss += loss.item()

      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
      total += pred.shape[0]
      perc = 100. * correct / total
      avg_loss = train_loss / (data_ct + 1)
      iter.set_description(f"[Train Epoch {epoch}] Total loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({perc:.2f}%)")

  def test(self, epoch):
    self.network.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      iter = tqdm(self.test_loader, total=len(self.test_loader))
      for data_ct, (data, target) in enumerate(iter):
        output = self.network(data)
        test_loss += self.loss(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        total += pred.shape[0]
        perc = 100. * correct / total
        avg_loss = test_loss / (data_ct + 1)
        iter.set_description(f"[Test Epoch {epoch}] Total loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({perc:.2f}%)")

  def train(self, n_epochs):
    self.test(0)
    for epoch in range(1, n_epochs + 1):
      self.train_epoch(epoch)
      self.test(epoch)


**Step 4** Train the model, and see the performance. :)

In [ ]:
def train_sum_3():
  # Feel free to modify the parameters here
  n_epochs=3
  learning_rate=0.001
  provenance="difftopkproofs"
  k=3

  trainer = Trainer(train_loader, test_loader, learning_rate, k, provenance)
  trainer.train(n_epochs)

train_sum_3()

**Step 5** 
Let's plot the confusion matrix for the neural network, and check the performance for single-digit recognition.

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy
import seaborn as sn
import pandas as pd

diagnose_batch_size = 32
mnist_diagnose_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True, transform=mnist_img_transform)
mnist_loader = torch.utils.data.DataLoader(mnist_diagnose_dataset, batch_size=diagnose_batch_size)

# Get prediction result
y_true, y_pred = [], []
with torch.no_grad():
    for (imgs, digits) in mnist_loader:
        pred_digits = numpy.argmax(trainer.network.mnist_net(imgs), axis=1)
        y_true += [d.item() for (i, d) in enumerate(digits)]
        y_pred += [d.item() for (i, d) in enumerate(pred_digits)]

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cm, index=list(range(10)), columns=list(range(10)))
plt.figure(figsize=(10,7))
sn.heatmap(df_cm, annot=True, cmap=plt.cm.Blues)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()

### P5: MNIST Sort 2
In this practice, we will learn the MNIST digit recognition through the sort 2 task. The task takes in two MNIST digits and returns 0 if the first digit is smaller than the second image, otherwise, returns 1.

**Step 1** Dataloader construction. 

First, we want to construct a train data loader, and a test data loader separately. 
The `get_item` function shall take in an index and return a tuple. The first tuple element is a tuple of tensorized images, and the second tuple element is 0 or 1.
The `collate_fn` function shall take in a list of tuples returned by `get_item`, and return a tuple. The first tuple element is tuples of batched tensors representing the images, and the second element is a tensor of batched 0 or 1s. 

In [ ]:

# TODO: Implent the MNISTSort2Dataset and the Dataloaders

**Step 2** Construct a classifier `MNISTSort2Net` that takes in two MNIST images and returns a tensor of the distribution over 0 and 1. You can utilize the previously defined class `MNISTNet`.

In [ ]:

# TODO: Implement the MNISTSort2Net

**Step 3** Setup trainer and loss function. We will use the BCE loss function for training the model.

In [ ]:
# TODO: Implement the loss function and Trainer

**Step 4** Train the model with different extended provenance semirings and check the results.

In [ ]:
# TODO: Perform model training

**Step 5** 
Please plot the confusion matrix for the neural network with different extended provenance semiring setups.
1. diffminmaxprob
2. difftopkproofs with k = 3
3. difftopkproofs with k = 10

In [ ]:
# TODO: Perform error analysis using confusion matrix 
# using the three extended provenance semirings mentioned above.